In [11]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import os

# Global figure template settings for a scientific look
plotly_scientific_template = dict(
    layout=go.Layout(
        font=dict(
            family="Serif, Times New Roman, Georgia",
            size=16,
            color="black"
        ),
        title_font=dict(
            family="Serif, Times New Roman, Georgia",
            size=20,
            color="black"
        ),
        paper_bgcolor='white',
        plot_bgcolor='white',
        xaxis=dict(
            showgrid=True,
            gridcolor='lightgrey',
            zeroline=False,
            linecolor='black',
            ticks='outside',
            ticklen=5,
            mirror=True
        ),
        yaxis=dict(
            showgrid=True,
            gridcolor='lightgrey',
            zeroline=False,
            linecolor='black',
            ticks='outside',
            ticklen=5,
            mirror=True
        ),
        colorway=['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd'],  # Scientific color palette
        legend=dict(
            bordercolor='black',
            borderwidth=1,
            bgcolor='white',
            font=dict(size=14)
        )
    )
)

# Register the template globally
import plotly.io as pio
pio.templates['scientific'] = plotly_scientific_template
pio.templates.default = 'scientific'

own_path = os.getcwd()
own_path = 'MAKai'

# get all sheet names
sheet_names = pd.ExcelFile(own_path+'\MasterarbeitenDatenAlleV2.xlsx').sheet_names

color_palette = {1: 'rgb(248, 246, 245)', 2: 'rgb(228, 227, 221)', 3: 'rgb(77, 75, 70)', 4: 'rgb(134, 0, 71)', 5: 'rgb(179, 6, 44)', 6: 'rgb(277, 186, 15)', 7: 'rgb(115, 124, 69)',
                 8: 'rgb(0, 97, 143)', 9: 'rgb(173, 59, 118)', 10: 'rgb(201, 98, 21)', 11: 'rgb(247, 217, 38)', 12: 'rgb(165, 171, 82)', 13: 'rgb(72, 169, 218)'}


dfs = []
for sheet_name in sheet_names:
    if sheet_name == 'Codierung':
        codes = pd.read_excel(own_path+'\MasterarbeitenDatenAlleV2.xlsx', sheet_name=sheet_name)
    else:
        df = pd.read_excel(own_path+'\MasterarbeitenDatenAlleV2.xlsx', sheet_name=sheet_name)
        df['DT'] = sheet_name
        dfs.append(df)

code_color = {
    0: 0,
    1: -1,
    2: -1,
    3: 0,
    4: 1,
    5: 0,
}

codes['code_color'] = codes['Codierung'].map(code_color)

# concatenate all dataframes
df = pd.concat(dfs, ignore_index=True)
df = df.set_index(['DT', 'Land']).melt(ignore_index=False).set_index('variable', append=True)
df = df.pivot_table(index=['Land', 'variable'], columns='DT', values='value')
df.index.names = ['Land', 'Jahr']
codes = codes.set_index('Land')
df = df.merge(codes, left_index=True, right_index=True).drop('Typ', axis=1)
df = df.set_index('Codierung', append=True).sort_index()

name_map = {
    'Kosten': ['Anteil  BIP Private', 'Anteil BIP Public ', 'Gesundheitsausgaben pro Kopf', 'Out of Pocket'],
    'Zugänglichkeit': ['Artzbesuche (pro Kopf)', 'Belegungsrate Akutpflegebet', 'Hospital beds', 'Practising doctors', 'Professional nurses'],
    'Qualität': ['Krebs M', 'Krebs W', 'Schlaganfall M', 'Schlaganfall W', 'Sterblichkeit ab 65 M', 'Sterblichkeit ab 65 W', 'Verhinderbare Sterblichkeitsrat']
}

score_map = {
    'Anteil  BIP Private': 1, 
    'Anteil BIP Public ': 1, 
    'Artzbesuche (pro Kopf)': 1,
    'Belegungsrate Akutpflegebet': -1, 
    'Gesundheitsausgaben pro Kopf': 1,
    'Hospital beds': 1, 
    'Krebs M': -1, 
    'Krebs W': -1, 
    'Out of Pocket': 1,
    'Practising doctors': 1,
    'Professional nurses': 1, 
    'Schlaganfall M': -1,
    'Schlaganfall W': -1,
    'Sterblichkeit ab 65 M': 1, 
    'Sterblichkeit ab 65 W': 1,
    'Verhinderbare Sterblichkeitsrat': -1
    }

# fill missing values
df = df.groupby(['Land']).ffill()

# z normalize over all years per variable
df = (df - df.mean()) / df.std()

# add columns for each category
for key in name_map.keys():
    tmp_sum = []
    for col in name_map[key]:
        tmp_sum.append(df[col] * score_map[col])

    tmp_sum = pd.concat(tmp_sum, axis=1).mean(axis=1)

    df[key] = tmp_sum

# filter year > 2010
df_f = df[df.index.get_level_values('Jahr') > 2010]

In [12]:
corr_data = df_f.copy()

corr_data['Krebs'] = (corr_data['Krebs M'] + corr_data['Krebs W']) / 2
corr_data['Schlaganfall'] = (corr_data['Schlaganfall M'] + corr_data['Schlaganfall W']) / 2
corr_data['Sterblichkeit'] = (corr_data['Sterblichkeit ab 65 M'] + corr_data['Sterblichkeit ab 65 W']) / 2
corr_data['Practising Medical Staff'] = (corr_data['Practising doctors'] + corr_data['Professional nurses']) / 2

corr_data = corr_data.drop(['Krebs M', 'Krebs W', 'Schlaganfall M', 'Schlaganfall W', 'Sterblichkeit ab 65 M', 'Sterblichkeit ab 65 W', 'Practising doctors', 'Professional nurses'], axis=1)
corr_data = corr_data[['Anteil  BIP Private', 'Anteil BIP Public ', 'Gesundheitsausgaben pro Kopf', 'Out of Pocket', 'Artzbesuche (pro Kopf)', 'Belegungsrate Akutpflegebet', 'Hospital beds', 'Practising Medical Staff', 'Krebs', 'Schlaganfall', 'Sterblichkeit', 'Verhinderbare Sterblichkeitsrat']]

colorscale = [[0, color_palette[13]], [0.5, color_palette[1]], [1, color_palette[5]]]

fig = px.imshow(corr_data.corr(), title='Correlation Matrix', labels=dict(x='Variable', y='Variable', color='Correlation'), color_continuous_scale=colorscale)
fig.update_layout(width=1000, height=1000)
fig.show()

In [5]:
# count data points per year
df.groupby('Jahr').count().mean(axis=1)


Jahr
2006     1.25
2007    10.40
2008    24.20
2009    18.75
2010    28.95
2011    29.90
2012    30.65
2013    32.00
2014    33.15
2015    33.35
2016    34.10
2017    34.75
2018    34.20
2019    34.30
2020    35.45
2021    32.70
2022    35.90
dtype: float64

In [40]:
plot_bar = df_f[list(name_map.keys())].reset_index()
color_seq = [color_palette[3], color_palette[4], color_palette[5], color_palette[7], color_palette[12], color_palette[13]]

fig = go.Figure()
fig = make_subplots(
    rows=2, cols=1,
)
tmp_codes = np.sort(plot_bar['Codierung'].unique())

for code in tmp_codes:
    plot_bar_tmp = plot_bar[plot_bar['Codierung'] == code]
    x = plot_bar_tmp['Jahr']
    fig.add_trace(go.Box(
        x=x,
        y=plot_bar_tmp['Kosten'],
        name=str(code),
        boxpoints=False,
        marker_color=color_seq[code]
    ),
        row=2, col=1
    )
    plot_line_tmp = plot_bar_tmp[['Jahr', 'Kosten']].groupby('Jahr').mean()
    fig.add_trace(go.Scatter(
        x=plot_line_tmp.index,
        y=plot_line_tmp['Kosten'],
        name=str(code),
        marker_color=color_seq[code],
        mode='lines',
    ),
        row=1, col=1
    )

fig.update_layout(
    yaxis=dict(
        title=dict(
            text='Kosten')
    ),
    boxmode='group' # group together boxes of the different traces for each value of x
)
fig.update_layout(
    legend=dict(
        orientation="h",
        yanchor="bottom",
        y=-0.3,
        xanchor="center",
        x=0.5
    ),
    title='Kosten',
    height=1000,
    width=1000
)
fig.show()

In [39]:
plot_bar_tmp = plot_bar[plot_bar['Codierung'] == 0]
plot_bar_line = plot_bar_tmp[['Jahr', 'Kosten']].groupby('Jahr').mean()

In [42]:
# plot time series grouped by DT
plt_data = df_f.groupby(['Jahr', 'Codierung']).mean()
err_data = df_f.groupby(['Jahr', 'Codierung']).std()

plt_data_lines = pd.concat([plt_data[list(name_map.keys())], err_data[list(name_map.keys())].add_prefix('err')], axis=1).reset_index()

for col in name_map.keys():
    fig = px.line(plt_data_lines, x='Jahr', y=col, color='Codierung', title=col, 
                  error_y='err'+col,
                  color_discrete_sequence=[color_palette[3], color_palette[4], color_palette[5], color_palette[7], color_palette[12], color_palette[13]])
    fig.update_layout(height=600, width=800)
    # more scientific layout
    fig.show()


In [7]:
plt_data_lines

Kosten  Zugänglichkeit  Qualität  errKosten  \
Jahr Codierung                                                  
2011 0         -0.367721       -0.391805 -0.982493   0.628920   
     1          0.138715       -0.095644  0.117454   0.311431   
     2          0.076277       -0.388695  0.035819   0.227611   
     3         -0.290576       -0.373497 -0.532522        NaN   
     4          0.396999        0.555045  0.290960   0.582117   
...                  ...             ...       ...        ...   
2022 1          0.383954        0.037476  0.592798   0.259538   
     2         -0.021244       -0.282684  0.588649   0.299064   
     3          0.070826       -0.033438 -0.002134        NaN   
     4          0.708031        0.334255  0.690418   0.942935   
     5         -0.056507        0.174441 -0.149890   0.582359   

                errZugänglichkeit  errQualität  
Jahr Codierung                                  
2011 0                   0.600462     0.789003  
     1                   0.432332     0.289030  
     2                   0.049677     0.278417  
     3                        NaN          NaN  
     4                   0.465718     0.266174  
...                           ...          ...  
2022 1                   0.478981     0.233299  
     2                   0.043368     0.066793  
     3                        NaN          NaN  
     4                   0.543152     0.330181  
     5                   0.415454     0.689955  

[72 rows x 6 columns]

In [46]:
# 3d scatter plot
plt_data3D = plt_data[np.logical_and(plt_data.index.get_level_values('Codierung') != 0,plt_data.index.get_level_values('Codierung')!= 3)].reset_index()

fig = px.scatter_3d(plt_data3D, x='Kosten', y='Zugänglichkeit', z='Qualität', color='Codierung')
fig.update_layout(width=800, height=800)
fig.show()



In [67]:
from plotly.subplots import make_subplots

# Create subplot layout (2x2)
fig = make_subplots(
    rows=2, cols=2,
    specs=[[{"type": "scatter"}, {"type": "scatter"}], 
           [{"type": "scatter"}, {"type": "scatter3d"}]],  # 3D in bottom right
    horizontal_spacing = 0.1,
    vertical_spacing = 0.1,
    subplot_titles=[
        "Kosten vs Zugänglichkeit", 
        "Kosten vs Qualität", 
        "Zugänglichkeit vs Qualität", 
        "Kosten vs Zugänglichkeit vs Qualität"
    ]
)

colorlist_3D = []
for i in plt_data3D["Codierung"]:
    colorlist_3D.append(color_seq[i])

# 2D Scatter plots
fig.add_trace(
    go.Scatter(
        x=plt_data3D["Kosten"], 
        y=plt_data3D["Zugänglichkeit"], 
        mode="markers", 
        marker=dict(color=colorlist_3D),
        name="Kosten vs Zugänglichkeit"
    ),
    row=1, col=1
)

fig.add_trace(
    go.Scatter(
        x=plt_data3D["Kosten"], 
        y=plt_data3D["Qualität"], 
        mode="markers", 
        marker=dict(color=colorlist_3D),
        name="Kosten vs Qualität"
    ),
    row=1, col=2
)

fig.add_trace(
    go.Scatter(
        x=plt_data3D["Zugänglichkeit"], 
        y=plt_data3D["Qualität"], 
        mode="markers", 
        marker=dict(color=colorlist_3D),
        name="Zugänglichkeit vs Qualität"
    ),
    row=2, col=1
)

# 3D Scatter plot
fig.add_trace(
    go.Scatter3d(
        x=plt_data3D["Kosten"], 
        y=plt_data3D["Zugänglichkeit"], 
        z=plt_data3D["Qualität"], 
        mode="markers", 
        marker=dict(size=4, color=colorlist_3D),
        name="Kosten vs Zugänglichkeit vs Qualität",
    ),
    row=2, col=2
)

fig.update_layout(
    scene4=dict(  # 'scene4' if the 3D plot is the 4th subplot
        domain=dict(x=[0.4, 1.0], y=[0.0, 0.5]),  # Adjust these to control the 3D plot size/position
        camera=dict(eye=dict(x=1.5, y=1.5, z=1.5))
    )
)

# Update layout
fig.update_layout(
    title="Multi-View Scatter Plots",
    width=1000, height=1000,
    showlegend=False,
    margin=dict(t=70, b=50)
)

# Show the plot
fig.show()

In [8]:
# use statsmodels to do a linear regression with fixed effects for year and group
import statsmodels.api as sm
import statsmodels.formula.api as smf

df_r = df_f.reset_index()
df_r['Jahr'] = df_r['Jahr'].astype('category')
df_r['Land'] = df_r['Land'].astype('category')
# drop codings 0 and 3
df_r = df_r[np.logical_and(df_r['Codierung'] != 0, df_r['Codierung'] != 3)]
df_r['Codierung'] = df_r['Codierung'].astype('category')
df_r['system'] = df_r['Codierung'].map(code_color)



# regression for quality_i,j = a + b * Kosten_i,j + c * Zugänglichkeit_i,j + d * Jahr_i + e * Codierung_i
model = smf.ols('Qualität ~ Kosten + Zugänglichkeit + C(Jahr) + C(Codierung)', data=df_r).fit()

print(model.summary())

                            OLS Regression Results                            
Dep. Variable:               Qualität   R-squared:                       0.606
Model:                            OLS   Adj. R-squared:                  0.580
Method:                 Least Squares   F-statistic:                     23.73
Date:                Wed, 19 Mar 2025   Prob (F-statistic):           3.92e-41
Time:                        20:59:10   Log-Likelihood:                -97.091
No. Observations:                 264   AIC:                             228.2
Df Residuals:                     247   BIC:                             289.0
Df Model:                          16                                         
Covariance Type:            nonrobust                                         
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
Intercept             0.0171      0.08

In [30]:
# regression for quality_i,j = a + b * Kosten_i,j + c * Zugänglichkeit_i,j + d * Jahr_i + e * Codierung_i
model = smf.ols('Zugänglichkeit ~ Qualität + Kosten + C(Jahr) + C(Codierung)', data=df_r).fit()

print(model.summary())

                            OLS Regression Results                            
Dep. Variable:         Zugänglichkeit   R-squared:                       0.475
Model:                            OLS   Adj. R-squared:                  0.441
Method:                 Least Squares   F-statistic:                     13.95
Date:                 Sa, 08 Mrz 2025   Prob (F-statistic):           1.88e-26
Time:                        14:33:56   Log-Likelihood:                -82.906
No. Observations:                 264   AIC:                             199.8
Df Residuals:                     247   BIC:                             260.6
Df Model:                          16                                         
Covariance Type:            nonrobust                                         
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
Intercept            -0.0924      0.07

In [34]:
# regression for quality_i,j = a + b * Kosten_i,j + c * Zugänglichkeit_i,j + d * Jahr_i + e * Codierung_i
model = smf.ols('Kosten ~ Zugänglichkeit + Qualität + C(Jahr) + C(Codierung)', data=df_r).fit()

print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                 Kosten   R-squared:                       0.473
Model:                            OLS   Adj. R-squared:                  0.439
Method:                 Least Squares   F-statistic:                     13.86
Date:                 Sa, 08 Mrz 2025   Prob (F-statistic):           2.63e-26
Time:                        14:34:15   Log-Likelihood:                -130.23
No. Observations:                 264   AIC:                             294.5
Df Residuals:                     247   BIC:                             355.2
Df Model:                          16                                         
Covariance Type:            nonrobust                                         
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
Intercept             0.1352      0.09

In [31]:
# regression for quality_i,j = a + b * Kosten_i,j + c * Zugänglichkeit_i,j + d * Jahr_i + e * Codierung_i
model = smf.ols('Qualität ~ Kosten + Zugänglichkeit + C(Jahr)', data=df_r).fit()

print(model.summary())

                            OLS Regression Results                            
Dep. Variable:               Qualität   R-squared:                       0.508
Model:                            OLS   Adj. R-squared:                  0.483
Method:                 Least Squares   F-statistic:                     19.88
Date:                 Sa, 08 Mrz 2025   Prob (F-statistic):           9.85e-32
Time:                        14:34:01   Log-Likelihood:                -126.26
No. Observations:                 264   AIC:                             280.5
Df Residuals:                     250   BIC:                             330.6
Df Model:                          13                                         
Covariance Type:            nonrobust                                         
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
Intercept          -0.0776      0.086     

In [32]:
# regression for quality_i,j = a + b * Kosten_i,j + c * Zugänglichkeit_i,j + d * Jahr_i + e * Codierung_i
model = smf.ols('Zugänglichkeit ~ Qualität + Kosten + C(Jahr)', data=df_r).fit()

print(model.summary())

                            OLS Regression Results                            
Dep. Variable:         Zugänglichkeit   R-squared:                       0.189
Model:                            OLS   Adj. R-squared:                  0.147
Method:                 Least Squares   F-statistic:                     4.493
Date:                 Sa, 08 Mrz 2025   Prob (F-statistic):           7.06e-07
Time:                        14:34:02   Log-Likelihood:                -140.16
No. Observations:                 264   AIC:                             308.3
Df Residuals:                     250   BIC:                             358.4
Df Model:                          13                                         
Covariance Type:            nonrobust                                         
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
Intercept           0.0806      0.091     

In [33]:
# regression for quality_i,j = a + b * Kosten_i,j + c * Zugänglichkeit_i,j + d * Jahr_i + e * Codierung_i
model = smf.ols('Kosten ~ Zugänglichkeit + Qualität + C(Jahr)', data=df_r).fit()

print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                 Kosten   R-squared:                       0.456
Model:                            OLS   Adj. R-squared:                  0.428
Method:                 Least Squares   F-statistic:                     16.12
Date:                 Sa, 08 Mrz 2025   Prob (F-statistic):           1.69e-26
Time:                        14:34:02   Log-Likelihood:                -134.44
No. Observations:                 264   AIC:                             296.9
Df Residuals:                     250   BIC:                             346.9
Df Model:                          13                                         
Covariance Type:            nonrobust                                         
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
Intercept           0.0763      0.089     

In [18]:
# test hypothesis:
# hypothesis: system 1 has higher quality than system 0
# hypothesis: system 1 has higher accessibility than system -1
# hypothesis: system 1 has higher costs than system 0
# hypothesis: system -1 has higher costs than system 0
# hypothesis: system -1 has higher quality than system 0

# hypothesis: system -1 has higher accessibility than system 0
# hypothesis: system 1 has higher quality than system -1
# hypothesis: system 1 has higher costs than system -1

# test hypothesis 1

import scipy.stats as stats

t_texts = [
    'system 1 has higher quality than system 0',
    'system 1 has higher accessibility than system -1',
    'system 1 has higher costs than system 0',
    'system -1 has higher costs than system 0',
    'system -1 has higher quality than system 0',
    'system -1 has higher accessibility than system 0',
    'system 1 has higher quality than system -1',
    'system 1 has higher costs than system -1'
]

t_res = []

t_res.append([stats.ttest_ind(df_r[df_r['system'] == 1]['Qualität'], df_r[df_r['system'] == 0]['Qualität']), df_r[df_r['system'] == 1]['Qualität'].mean(), df_r[df_r['system'] == 0]['Qualität'].mean()])
t_res.append([stats.ttest_ind(df_r[df_r['system'] == 1]['Zugänglichkeit'], df_r[df_r['system'] == -1]['Zugänglichkeit']), df_r[df_r['system'] == 1]['Zugänglichkeit'].mean(), df_r[df_r['system'] == -1]['Zugänglichkeit'].mean()])
t_res.append([stats.ttest_ind(df_r[df_r['system'] == 1]['Kosten'], df_r[df_r['system'] == 0]['Kosten']), df_r[df_r['system'] == 1]['Kosten'].mean(), df_r[df_r['system'] == 0]['Kosten'].mean()])
t_res.append([stats.ttest_ind(df_r[df_r['system'] == -1]['Kosten'], df_r[df_r['system'] == 0]['Kosten']), df_r[df_r['system'] == -1]['Kosten'].mean(), df_r[df_r['system'] == 0]['Kosten'].mean()])
t_res.append([stats.ttest_ind(df_r[df_r['system'] == -1]['Qualität'], df_r[df_r['system'] == 0]['Qualität']), df_r[df_r['system'] == -1]['Qualität'].mean(), df_r[df_r['system'] == 0]['Qualität'].mean()])
t_res.append([stats.ttest_ind(df_r[df_r['system'] == -1]['Zugänglichkeit'], df_r[df_r['system'] == 0]['Zugänglichkeit']), df_r[df_r['system'] == -1]['Zugänglichkeit'].mean(), df_r[df_r['system'] == 0]['Zugänglichkeit'].mean()])
t_res.append([stats.ttest_ind(df_r[df_r['system'] == 1]['Qualität'], df_r[df_r['system'] == -1]['Qualität']), df_r[df_r['system'] == 1]['Qualität'].mean(), df_r[df_r['system'] == -1]['Qualität'].mean()])
t_res.append([stats.ttest_ind(df_r[df_r['system'] == 1]['Kosten'], df_r[df_r['system'] == -1]['Kosten']), df_r[df_r['system'] == 1]['Kosten'].mean(), df_r[df_r['system'] == -1]['Kosten'].mean()])

for res in t_res:
    print(f'{t_texts[t_res.index(res)]}: p-value: {round(res[0].pvalue,3)}, mean 1: {round(res[1],3)}, mean 2: {round(res[2],3)}')

system 1 has higher quality than system 0: p-value: 0.0, mean 1: 0.531, mean 2: -0.255
system 1 has higher accessibility than system -1: p-value: 0.0, mean 1: 0.496, mean 2: -0.193
system 1 has higher costs than system 0: p-value: 0.0, mean 1: 0.564, mean 2: -0.183
system -1 has higher costs than system 0: p-value: 0.0, mean 1: 0.229, mean 2: -0.183
system -1 has higher quality than system 0: p-value: 0.0, mean 1: 0.414, mean 2: -0.255
system -1 has higher accessibility than system 0: p-value: 0.0, mean 1: -0.193, mean 2: 0.246
system 1 has higher quality than system -1: p-value: 0.01, mean 1: 0.531, mean 2: 0.414
system 1 has higher costs than system -1: p-value: 0.0, mean 1: 0.564, mean 2: 0.229
